In [1]:
import pandas as pd
from pathlib import Path
import os, yaml
if '_fixed' not in locals():
    _fixed = os.chdir(os.path.dirname(os.getcwd()))
from ludwig import repo_root
from ludwig.util import vllm_Client
from collections import Counter
from tqdm.notebook import tqdm
import random
from tabulate import tabulate
import json
import networkx as nx
import io
import matplotlib.pyplot as plt

In [2]:

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/livebench/reasoning/data/test-00000-of-00001.parquet")

In [3]:
df.head()

,question_id,category,ground_truth,turns,task,livebench_release_date,livebench_removal_date,level
0,bd14ba82d1fe813dcdf6c16f0f868e5f3fe92e4cee9cf5...,reasoning,"1, filmmaking, police-officer, journalist",[There are 2 people standing in a line. From l...,zebra_puzzle,2024-11-25,,15.0
1,20d48b97e524cd64f82b0e4fb3c597a37bae1d9357e9b2...,reasoning,"freelancer, animation, thriller, 1",[There are 2 people standing in a line. From l...,zebra_puzzle,2024-11-25,,16.0
2,b1b1678a4f290c8ab2b85f957a0cdbba93ef0c97d1e7ab...,reasoning,"architect, skiing, musical, 2",[There are 2 people standing in a line. From l...,zebra_puzzle,2024-11-25,,17.0
3,743aa0ae2d518d95d45ab8b77a7e9ffde5a03c07da8a6a...,reasoning,"canadian, 2, chef, thai",[There are 2 people standing in a line. From l...,zebra_puzzle,2024-11-25,,14.0
4,84650f1042abf3d267bb79b6d1e1d899d27327a11ab81f...,reasoning,"1, fisherman, coffee, musician",[There are 2 people standing in a line. From l...,zebra_puzzle,2024-11-25,,12.0


In [3]:
df['task'].value_counts()

task
zebra_puzzle      100
spatial            50
web_of_lies_v2     50
Name: count, dtype: int64

In [ ]:
client = vllm_Client('8001')
client.prepare()

In [65]:
x = df.sample()
turns = x['turns'].values[0].tolist()
gt = x['ground_truth'].values[0]
print(turns[0])
gt

There are 4 people standing in a line numbered 1 through 4 in a left to right order.
Each person has a set of attributes: Pet, Movie-Genre, Sport, Beverage.
The attributes have the following possible values:
- Pet: frog, rabbit, horse, lizard
- Movie-Genre: action movies, drama movies, family movies, spy movies
- Sport: rugby, water-polo, volleyball, golf
- Beverage: juice, fanta, tea, sprite
and exactly one person in the line has a given value for an attribute.

Given the following premises about the line of people:
- the person that has a rabbit drinks sprite or the person that watches drama movies has a rabbit or both
- the person that watches drama movies plays volleyball or the person that plays volleyball drinks juice or both
- the person that has a lizard is not anywhere to the right of the person that plays water-polo
- the person who drinks fanta is not anywhere to the right of the person that plays golf
- the person that plays volleyball is somewhere between the person that p

'rabbit'

In [68]:
resp = client.step(turns[0], max_tokens=4096)

msg = resp['choices'][0]['message']
print(msg['content'])

***rabbit***


In [ ]:
resp